In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [7]:
W = torch.zeros( (2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# hyp = 1 / (1+torch.exp(-(x_train.matmul(W)+b)))
hyp = torch.sigmoid(x_train.matmul(W)+b)    

# cost = (-( y_train*torch.log(hyp) + (1-y_train)*(1-torch.log(hyp)) )).mean()
cost = F.binary_cross_entropy(hyp, y_train)
print(cost)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward>)


In [14]:
# simple 1-variable Logistic Regression

x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

W = torch.zeros( (2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = torch.optim.SGD([W, b], lr=1)
nb_epoch = 1000
for epoch in range(nb_epoch) :
  hyp = torch.sigmoid(x_train.matmul(W)+ b)
  cost = F.binary_cross_entropy(hyp, y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()
  if epoch % 200 == 0:
      print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epoch, cost.item()
        ))

# test
hyp = torch.sigmoid(x_train.matmul(W)+b)
prediction = hyp >= torch.FloatTensor([0.5])
print(y_train)
print(prediction)

Epoch    0/1000 Cost: 0.693147
Epoch  200/1000 Cost: 0.080643
Epoch  400/1000 Cost: 0.045300
Epoch  600/1000 Cost: 0.031672
Epoch  800/1000 Cost: 0.024394
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])
tensor([[False],
        [False],
        [False],
        [ True],
        [ True],
        [ True]])


In [22]:
# Logistic Regression with nn Module

model = nn.Sequential(nn.Linear(2,1), nn.Sigmoid())   ## 차례로 쌓는다. Wx+b 다음에 sigmoid
model(x_train)    # 처음 W와 b는 random


optimizer = torch.optim.SGD(model.parameters(), lr=1)

nb_epoch = 1000
for epoch in range(nb_epoch) :
  y_predict = model(x_train)
  cost = F.binary_cross_entropy(y_predict, y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()
  if epoch % 100 == 0:
      prediction = y_predict >= torch.FloatTensor([0.5]) # 예측값이 0.5를 넘으면 True로 간주
      correct_prediction = prediction.float() == y_train # 실제값과 일치하는 경우만 True로 간주
      accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
      print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format( # 각 에포크마다 정확도를 출력
          epoch, nb_epoch, cost.item(), accuracy * 100,
      ))


Epoch    0/1000 Cost: 0.841201 Accuracy 50.00%
Epoch  100/1000 Cost: 0.131552 Accuracy 100.00%
Epoch  200/1000 Cost: 0.079525 Accuracy 100.00%
Epoch  300/1000 Cost: 0.057331 Accuracy 100.00%
Epoch  400/1000 Cost: 0.044954 Accuracy 100.00%
Epoch  500/1000 Cost: 0.037029 Accuracy 100.00%
Epoch  600/1000 Cost: 0.031505 Accuracy 100.00%
Epoch  700/1000 Cost: 0.027430 Accuracy 100.00%
Epoch  800/1000 Cost: 0.024296 Accuracy 100.00%
Epoch  900/1000 Cost: 0.021809 Accuracy 100.00%


In [25]:
# Logistc Regression with Class

class BinaryClassifier(nn.Module) :
  def __init__(self) :
    super().__init__()
    self.linear = nn.Linear(2,1)
    self.sigmoid = nn.Sigmoid()
  
  def forward(self, x) :
    return self.sigmoid(self.linear(x))

model = BinaryClassifier()
optimizer = torch.optim.SGD(model.parameters(), lr=1)

nb_epoch = 2000
for epoch in range(nb_epoch) :
  hyp = model(x_train)
  cost = F.binary_cross_entropy(hyp, y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 400 == 0:
    prediction = hyp>= torch.FloatTensor([0.5]) # 예측값이 0.5를 넘으면 True로 간주
    correct_prediction = prediction.float() == y_train # 실제값과 일치하는 경우만 True로 간주
    accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
    print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format( # 각 에포크마다 정확도를 출력
        epoch, nb_epoch, cost.item(), accuracy * 100,
    ))


Epoch    0/2000 Cost: 0.595946 Accuracy 66.67%
Epoch  400/2000 Cost: 0.045899 Accuracy 100.00%
Epoch  800/2000 Cost: 0.024564 Accuracy 100.00%
Epoch 1200/2000 Cost: 0.016822 Accuracy 100.00%
Epoch 1600/2000 Cost: 0.012801 Accuracy 100.00%
